# 1. 首先讲bevfusion中的get_bboxes方法摘出来
- 1. ffn的结果存成pkl
- 2.

In [ ]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
with open("ffn_res.pkl", "rb") as f1:
    pred_dict = pickle.load(f1)
for i in pred_dict[0][0]:
    pred_dict[0][0][i] = pred_dict[0][0][i].to("cpu").float()

In [3]:
with open("self.pkl", "rb") as f2:
    self = pickle.load(f2)
self = self.to("cpu")
self.query_labels = self.query_labels.to("cpu")

In [4]:
with open("metas.pkl", "rb") as f3:
    metas = pickle.load(f3)

In [5]:
def decode(heatmap, rot, dim, center, height, vel, filter=False):
    """Decode bboxes.
    Args:
        heat (torch.Tensor): Heatmap with the shape of [B, num_cls, num_proposals].
        rot (torch.Tensor): Rotation with the shape of
            [B, 1, num_proposals].
        dim (torch.Tensor): Dim of the boxes with the shape of
            [B, 3, num_proposals].
        center (torch.Tensor): bev center of the boxes with the shape of
            [B, 2, num_proposals]. (in feature map metric)
        hieght (torch.Tensor): height of the boxes with the shape of
            [B, 2, num_proposals]. (in real world metric)
        vel (torch.Tensor): Velocity with the shape of [B, 2, num_proposals].
        filter: if False, return all box without checking score and center_range
    Returns:
        list[dict]: Decoded boxes.
    """
    # my add code 1111111111 我自己加的代码
    self.out_size_factor = 8
    self.voxel_size = [0.075, 0.075]
    self.pc_range = [-54.0, -54.0]
    self.score_threshold = 0.0
    self.post_center_range = torch.tensor([-61.2000, -61.2000, -10.0000,  61.2000,  61.2000,  10.0000])
    
    # class label
    final_preds = heatmap.max(1, keepdims=False).indices
    final_scores = heatmap.max(1, keepdims=False).values

    # change size to real world metric
    center[:, 0, :] = center[:, 0, :] * self.out_size_factor * self.voxel_size[0] + self.pc_range[0]
    center[:, 1, :] = center[:, 1, :] * self.out_size_factor * self.voxel_size[1] + self.pc_range[1]
    # center[:, 2, :] = center[:, 2, :] * (self.post_center_range[5] - self.post_center_range[2]) + self.post_center_range[2]
    dim[:, 0, :] = dim[:, 0, :].exp()
    dim[:, 1, :] = dim[:, 1, :].exp()
    dim[:, 2, :] = dim[:, 2, :].exp()
    height = height - dim[:, 2:3, :] * 0.5  # gravity center to bottom center
    rots, rotc = rot[:, 0:1, :], rot[:, 1:2, :]
    rot = torch.atan2(rots, rotc)

    if vel is None:
        final_box_preds = torch.cat([center, height, dim, rot], dim=1).permute(0, 2, 1)
    else:
        final_box_preds = torch.cat([center, height, dim, rot, vel], dim=1).permute(0, 2, 1)

    predictions_dicts = []
    for i in range(heatmap.shape[0]):
        boxes3d = final_box_preds[i]
        scores = final_scores[i]
        labels = final_preds[i]
        predictions_dict = {
            'bboxes': boxes3d,
            'scores': scores,
            'labels': labels
        }
        predictions_dicts.append(predictions_dict)

    if filter is False:
        return predictions_dicts

    # use score threshold
    if self.score_threshold is not None:
        thresh_mask = final_scores > self.score_threshold

    if self.post_center_range is not None:
        self.post_center_range = torch.tensor(
            self.post_center_range, device=heatmap.device)
        mask = (final_box_preds[..., :3] >=
                self.post_center_range[:3]).all(2)
        mask &= (final_box_preds[..., :3] <=
                    self.post_center_range[3:]).all(2)

        predictions_dicts = []
        for i in range(heatmap.shape[0]):
            cmask = mask[i, :]
            if self.score_threshold:
                cmask &= thresh_mask[i]

            boxes3d = final_box_preds[i, cmask]
            scores = final_scores[i, cmask]
            labels = final_preds[i, cmask]
            predictions_dict = {
                'bboxes': boxes3d,
                'scores': scores,
                'labels': labels
            }

            predictions_dicts.append(predictions_dict)
    else:
        raise NotImplementedError(
            'Need to reorganize output as a batch, only '
            'support post_center_range is not None for now!')

    return predictions_dicts

In [6]:
def get_bboxes(self, preds_dicts, metas, img=None, rescale=False, for_roi=False):
        """Generate bboxes from bbox head predictions.
        Args:
            preds_dicts (tuple[list[dict]]): Prediction results.
        Returns:
            list[list[dict]]: Decoded bbox, scores and labels for each layer & each batch
        """
        rets = []
        for layer_id, preds_dict in enumerate(preds_dicts):
            batch_size = preds_dict[0]["heatmap"].shape[0]
            batch_score = preds_dict[0]["heatmap"][..., -self.num_proposals :].float().sigmoid()
            # if self.loss_iou.loss_weight != 0:
            #    batch_score = torch.sqrt(batch_score * preds_dict[0]['iou'][..., -self.num_proposals:].sigmoid())
            one_hot = F.one_hot(
                self.query_labels, num_classes=self.num_classes
            ).permute(0, 2, 1)
            batch_score = batch_score * preds_dict[0]["query_heatmap_score"] * one_hot

            batch_center = preds_dict[0]["center"][..., -self.num_proposals :]
            batch_height = preds_dict[0]["height"][..., -self.num_proposals :]
            batch_dim = preds_dict[0]["dim"][..., -self.num_proposals :]
            batch_rot = preds_dict[0]["rot"][..., -self.num_proposals :]
            batch_vel = None
            if "vel" in preds_dict[0]:
                batch_vel = preds_dict[0]["vel"][..., -self.num_proposals :]

            # temp = self.bbox_coder.decode(
            #     batch_score,
            #     batch_rot,
            #     batch_dim,
            #     batch_center,
            #     batch_height,
            #     batch_vel,
            #     filter=True,
            # )
            temp = decode(
                batch_score,
                batch_rot,
                batch_dim,
                batch_center,
                batch_height,
                batch_vel,
                filter=True,
            )
            if self.test_cfg["dataset"] == "nuScenes":
                self.tasks = [
                    dict(
                        num_class=8,
                        class_names=[],
                        indices=[0, 1, 2, 3, 4, 5, 6, 7],
                        radius=-1,
                    ),
                    dict(
                        num_class=1,
                        class_names=["pedestrian"],
                        indices=[8],
                        radius=0.175,
                    ),
                    dict(
                        num_class=1,
                        class_names=["traffic_cone"],
                        indices=[9],
                        radius=0.175,
                    ),
                ]
            elif self.test_cfg["dataset"] == "Waymo":
                self.tasks = [
                    dict(num_class=1, class_names=["Car"], indices=[0], radius=0.7),
                    dict(
                        num_class=1, class_names=["Pedestrian"], indices=[1], radius=0.7
                    ),
                    dict(num_class=1, class_names=["Cyclist"], indices=[2], radius=0.7),
                ]

            ret_layer = []
            for i in range(batch_size):
                boxes3d = temp[i]["bboxes"]
                scores = temp[i]["scores"]
                labels = temp[i]["labels"]
                ## adopt circle nms for different categories
                if self.test_cfg["nms_type"] != None:
                    keep_mask = torch.zeros_like(scores)
                    for task in self.tasks:
                        task_mask = torch.zeros_like(scores)
                        for cls_idx in task["indices"]:
                            task_mask += labels == cls_idx
                        task_mask = task_mask.bool()
                        if task["radius"] > 0:
                            if self.test_cfg["nms_type"] == "circle":
                                boxes_for_nms = torch.cat(
                                    [
                                        boxes3d[task_mask][:, :2],
                                        scores[:, None][task_mask],
                                    ],
                                    dim=1,
                                )
                                task_keep_indices = torch.tensor(
                                    circle_nms(
                                        boxes_for_nms.detach().cpu().numpy(),
                                        task["radius"],
                                    )
                                )
                            else:
                                boxes_for_nms = xywhr2xyxyr(
                                    metas[i]["box_type_3d"](
                                        boxes3d[task_mask][:, :7], 7
                                    ).bev
                                )
                                top_scores = scores[task_mask]
                                task_keep_indices = nms_gpu(
                                    boxes_for_nms,
                                    top_scores,
                                    thresh=task["radius"],
                                    pre_maxsize=self.test_cfg["pre_maxsize"],
                                    post_max_size=self.test_cfg["post_maxsize"],
                                )
                        else:
                            task_keep_indices = torch.arange(task_mask.sum())
                        if task_keep_indices.shape[0] != 0:
                            keep_indices = torch.where(task_mask != 0)[0][
                                task_keep_indices
                            ]
                            keep_mask[keep_indices] = 1
                    keep_mask = keep_mask.bool()
                    ret = dict(
                        bboxes=boxes3d[keep_mask],
                        scores=scores[keep_mask],
                        labels=labels[keep_mask],
                    )
                else:  # no nms
                    ret = dict(bboxes=boxes3d, scores=scores, labels=labels)
                ret_layer.append(ret)
            rets.append(ret_layer)
        assert len(rets) == 1
        assert len(rets[0]) == 1
        res = [
            [
                metas[0]["box_type_3d"](
                    rets[0][0]["bboxes"], box_dim=rets[0][0]["bboxes"].shape[-1]
                ),
                rets[0][0]["scores"],
                rets[0][0]["labels"].int(),
            ]
        ]
        return res

In [7]:
bboxes = get_bboxes(self, pred_dict, metas)
bboxes

/tmp/ipykernel_48940/1753587228.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.post_center_range = torch.tensor(


[[LiDARInstance3DBoxes(
      tensor([[-1.0636e+01,  9.7453e+00, -6.8970e-01,  ..., -4.3752e-02,
            8.1299e-02, -1.3730e+00],
          [ 5.6495e+00, -3.6696e+00, -9.1542e-01,  ..., -3.1334e+00,
           -1.3794e-02,  1.2012e+00],
          [ 9.1355e+00,  2.4821e+01, -2.4599e-01,  ...,  1.0290e-01,
           -1.8604e-01, -1.4609e+00],
          ...,
          [ 9.9743e+00,  2.5479e+01,  2.4265e-01,  ...,  9.1606e-01,
           -2.5000e-01, -3.3521e-01],
          [ 7.0275e+00, -4.3008e+00, -1.0529e+00,  ...,  3.0318e+00,
           -7.3120e-02,  3.9746e-01],
          [-1.2271e+01,  2.0570e+01, -8.0998e-01,  ...,  1.7608e+00,
            1.9257e-02,  3.7048e-02]])),
  tensor([7.8372e-01, 6.6182e-01, 7.5650e-01, 7.3698e-01, 7.9196e-01, 0.0000e+00,
          6.2229e-01, 5.9166e-01, 0.0000e+00, 1.5395e-01, 6.6446e-01, 9.9265e-08,
          7.3155e-01, 5.1333e-01, 1.8540e-06, 4.4399e-01, 1.5832e-02, 4.3789e-01,
          2.1567e-05, 5.0708e-01, 1.8826e-01, 1.9047e-06, 0.0000e+